## Import

In [1]:
# Data Librairies
import pandas as pd
import numpy as np

# Data Modeling
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb


# Data Vizualization Librairies
import seaborn as sns

# Warnings
import warnings
warnings.filterwarnings("ignore")

## Fonctions utilitaires

In [2]:
def getAgeCat(age: int) -> str:
    cat: str = ""

    if(age < 35):
        cat = "34 & -"
    elif (age >=35 and age < 45):
        cat = "35-44"
    elif (age >= 45 and age < 55):
        cat = "45-54"
    elif (age >= 55 and age < 65):
        cat = "55-64"
    else:
        cat = "65 & +"

    return cat

### Load Dataset

In [3]:
df = pd.read_csv("./Data/data_participant.csv", index_col=None)
df.describe()

,ID,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default-payment-next-month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,24000.000000
mean,15000.500000,167484.322667,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100,51223.330900,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.252333
std,8660.398374,129747.661567,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988,73635.860576,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.434361
min,1.000000,10000.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3558.750000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22381.500000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67091.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,1.000000
max,30000.000000,1000000.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [4]:
# Create "Remaining Amount" Column
df["RMN_AMT2"] = df["BILL_AMT2"] - df["PAY_AMT1"]
df["RMN_AMT3"] = df["BILL_AMT3"] - df["PAY_AMT2"]
df["RMN_AMT4"] = df["BILL_AMT4"] - df["PAY_AMT3"]
df["RMN_AMT5"] = df["BILL_AMT5"] - df["PAY_AMT4"]
df["RMN_AMT6"] = df["BILL_AMT6"] - df["PAY_AMT5"]

# Create "Sufficient Payment"
df["SUFFICIENT_PAYMENT_AMT2"] = np.where(df["RMN_AMT2"] <= 0, 1, 0)
df["SUFFICIENT_PAYMENT_AMT3"] = np.where(df["RMN_AMT3"] <= 0, 1, 0)
df["SUFFICIENT_PAYMENT_AMT4"] = np.where(df["RMN_AMT4"] <= 0, 1, 0)
df["SUFFICIENT_PAYMENT_AMT5"] = np.where(df["RMN_AMT5"] <= 0, 1, 0)
df["SUFFICIENT_PAYMENT_AMT6"] = np.where(df["RMN_AMT6"] <= 0, 1, 0)

# Add weight to most recent months
df["SUFFICIENT_PAYMENT_AMT2"] = df["SUFFICIENT_PAYMENT_AMT2"]*1.4
df["SUFFICIENT_PAYMENT_AMT3"] = df["SUFFICIENT_PAYMENT_AMT3"]*1.3
df["SUFFICIENT_PAYMENT_AMT4"] = df["SUFFICIENT_PAYMENT_AMT4"]*1.2
df["SUFFICIENT_PAYMENT_AMT5"] = df["SUFFICIENT_PAYMENT_AMT5"]*1.1
df["SUFFICIENT_PAYMENT_AMT6"] = df["SUFFICIENT_PAYMENT_AMT6"]*1

In [5]:
df.describe(exclude=np.number) # Describe Categorical Values

,SEX,EDUCATION,MARRIAGE
count,30000,30000,30000
unique,2,5,4
top,female,university,single
freq,18112,14030,15964


In [6]:
cats = df.select_dtypes(exclude=np.number).columns.tolist()
for cat in cats:
    df[cat] = df[cat].astype('category')
df.dtypes

ID                               int64
LIMIT_BAL                        int64
SEX                           category
EDUCATION                     category
MARRIAGE                      category
AGE                              int64
PAY_0                            int64
PAY_2                            int64
PAY_3                            int64
PAY_4                            int64
PAY_5                            int64
PAY_6                            int64
BILL_AMT1                        int64
BILL_AMT2                        int64
BILL_AMT3                        int64
BILL_AMT4                        int64
BILL_AMT5                        int64
BILL_AMT6                        int64
PAY_AMT1                         int64
PAY_AMT2                         int64
PAY_AMT3                         int64
PAY_AMT4                         int64
PAY_AMT5                         int64
PAY_AMT6                         int64
default-payment-next-month     float64
RMN_AMT2                 

In [7]:
# Creating Target and Explicative Datasets
df_training = df[(~df['default-payment-next-month'].isna())]
df_prediction = df[(df['default-payment-next-month'].isna())]

X, y = df_training.drop(['ID', 'default-payment-next-month'], axis=1), df_training['default-payment-next-month']


print(df_training.columns.tolist()) 

['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default-payment-next-month', 'RMN_AMT2', 'RMN_AMT3', 'RMN_AMT4', 'RMN_AMT5', 'RMN_AMT6', 'SUFFICIENT_PAYMENT_AMT2', 'SUFFICIENT_PAYMENT_AMT3', 'SUFFICIENT_PAYMENT_AMT4', 'SUFFICIENT_PAYMENT_AMT5', 'SUFFICIENT_PAYMENT_AMT6']


In [20]:
# Finding Protected Groups
df['group'] = df.apply(lambda x: x['SEX'] + '-' + x['EDUCATION'] + '-' + x['MARRIAGE'], axis=1)
noDefault = df[df['default-payment-next-month'] == 0].groupby(['group'])['default-payment-next-month'].count().rename("noDefault")
default = df[df['default-payment-next-month'] == 1].groupby(['group'])['default-payment-next-month'].count().rename("default")
df_groups = pd.concat([noDefault, default], axis=1).fillna(0)
df_groups["total"] = df_groups.apply(lambda x : x["noDefault"] + x["default"], axis=1)
df_groups["propDefault"] = df_groups.apply(lambda x : x["default"]/x["total"], axis=1)
propDefault_mean = df_groups["propDefault"].mean()
propDefault_median = df_groups["propDefault"].median()

0.19840539711744865

## Creating Train and Test Datasets

In [24]:
# Standard Sklearn Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)

In [25]:
# Defining Hyperparameters
params = {
    "objective": "binary:logistic",
    # "tree_method": "gpu_hist"
}

In [26]:
# Defining and Training Model
n = 100
model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
)

In [27]:
# RMSE Validation
preds = model.predict(dtest_reg)
rmse = mean_squared_error(y_test, preds, squared=False)

print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 0.405


In [28]:
# F1_Score Validation

In [29]:
# Faireness Validation